In [14]:
import pandas as pd

In [15]:
gca_df = pd.read_csv('data/gca_player_stats.csv')
defense_df = pd.read_csv('data/defense_player_stats.csv')
misc_df = pd.read_csv('data/misc_player_stats.csv')
passing_df = pd.read_csv('data/passing_player_stats.csv')
possession_df = pd.read_csv('data/possession_player_stats.csv')
shooting_df = pd.read_csv('data/shooting_player_stats.csv')


In [16]:
all_dfs = [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df]

for i in range(len(all_dfs)):
    all_dfs[i].columns = all_dfs[i].iloc[0]
    all_dfs[i] = all_dfs[i].iloc[1:].reset_index(drop=True)

In [22]:
for df in all_dfs:
    df = df.loc[:, df.columns.notna()]

In [18]:
for df in all_dfs:
    df.drop(columns=['Rk'], inplace=True)

In [23]:
possession_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,...,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches,NaN,NaN
0,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,...,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches,NaN,NaN
1,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,1.0,73,2,...,1,0,0,1,0,40,3,Matches,Premier League,possession
2,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.9,137,7,...,0,2,0,2,0,111,0,Matches,Premier League,possession
3,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,21.8,1337,62,...,14,22,1,28,12,757,10,Matches,Premier League,possession
4,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,15.7,1330,175,...,5,2,0,4,0,1000,1,Matches,Premier League,possession


In [20]:
def clean(df):
       df = df.drop_duplicates(subset=['Squad', 'Player'])
       return df.reset_index(drop=True)

gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df = \
       map(clean, [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df])
       


In [26]:
from functools import reduce

# List your category DataFrames
dfs = [shooting_df, passing_df, gca_df, defense_df, possession_df, misc_df]

# Clean each DataFrame: drop columns that are in the merge keys or will collide
cleaned = []
for df in dfs:
    # Keep only the merge keys + the metric columns (drop 'Nation','Pos', etc.)
    cols_to_keep = ['Squad', 'Player'] + [c for c in df.columns if c not in ['Squad','Player','Nation','Pos','Rk']]
    cleaned.append(df[cols_to_keep])

# Now perform a left join chain
merged = reduce(
    lambda left, right: left.merge(right, on=['Squad','Player'], how='left'),
    cleaned
)


MergeError: Passing 'suffixes' which cause duplicate columns {'Matches_x', '90s_x', 'Born_x', 'Age_x', 'nan_x'} is not allowed.

In [24]:
possession_df.to_csv('possession_player_stats.csv', index=False)